# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5717fdc2b0>
├── 'a' --> tensor([[ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532]])
└── 'x' --> <FastTreeValue 0x7f57acffa550>
    └── 'c' --> tensor([[-0.9505, -0.5114, -0.0539, -0.3478],
                        [-0.7235, -1.0530, -0.3435, -0.7083],
                        [ 0.2801,  0.2255, -1.6126,  1.3294]])

In [4]:
t.a

tensor([[ 2.3972,  0.7236,  0.7686],
        [ 0.7798, -0.7067, -1.1532]])

In [5]:
%timeit t.a

67.5 ns ± 0.0531 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5717fdc2b0>
├── 'a' --> tensor([[-0.3798,  0.2980,  0.3797],
│                   [-0.2746, -0.3837, -0.0434]])
└── 'x' --> <FastTreeValue 0x7f57acffa550>
    └── 'c' --> tensor([[-0.9505, -0.5114, -0.0539, -0.3478],
                        [-0.7235, -1.0530, -0.3435, -0.7083],
                        [ 0.2801,  0.2255, -1.6126,  1.3294]])

In [7]:
%timeit t.a = new_value

72.6 ns ± 0.0423 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.3972,  0.7236,  0.7686],
               [ 0.7798, -0.7067, -1.1532]]),
    x: Batch(
           c: tensor([[-0.9505, -0.5114, -0.0539, -0.3478],
                      [-0.7235, -1.0530, -0.3435, -0.7083],
                      [ 0.2801,  0.2255, -1.6126,  1.3294]]),
       ),
)

In [10]:
b.a

tensor([[ 2.3972,  0.7236,  0.7686],
        [ 0.7798, -0.7067, -1.1532]])

In [11]:
%timeit b.a

58.4 ns ± 0.11 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9156, -0.9557,  0.1892],
               [-1.0654,  0.8652, -0.7778]]),
    x: Batch(
           c: tensor([[-0.9505, -0.5114, -0.0539, -0.3478],
                      [-0.7235, -1.0530, -0.3435, -0.7083],
                      [ 0.2801,  0.2255, -1.6126,  1.3294]]),
       ),
)

In [13]:
%timeit b.a = new_value

500 ns ± 0.0521 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.913 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 554 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5706230bb0>
├── 'a' --> tensor([[[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]],
│           
│                   [[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]],
│           
│                   [[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]],
│           
│                   [[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]],
│           
│                   [[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]],
│           
│                   [[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]],
│           
│                   [[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]],
│           
│                   [[ 2.3972,  0.7236,  0.7686],
│                    [ 0.7798, -0.7067, -1.1532]]])
└── 'x' --> <FastTreeValue 0x7f5706230730>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.5 µs ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f57acfe5fd0>
├── 'a' --> tensor([[ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532],
│                   [ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532],
│                   [ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532],
│                   [ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532],
│                   [ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532],
│                   [ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532],
│                   [ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532],
│                   [ 2.3972,  0.7236,  0.7686],
│                   [ 0.7798, -0.7067, -1.1532]])
└── 'x' --> <FastTreeValue 0x7f57062ab430>
    └── 'c' --> tensor([[-0.9505, -0.5114, -0.0539, -0.3478],
                        [-0.7235, -1.0530, -0.3435, -0.7083],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 47.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 67.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.9505, -0.5114, -0.0539, -0.3478],
                       [-0.7235, -1.0530, -0.3435, -0.7083],
                       [ 0.2801,  0.2255, -1.6126,  1.3294]],
              
                      [[-0.9505, -0.5114, -0.0539, -0.3478],
                       [-0.7235, -1.0530, -0.3435, -0.7083],
                       [ 0.2801,  0.2255, -1.6126,  1.3294]],
              
                      [[-0.9505, -0.5114, -0.0539, -0.3478],
                       [-0.7235, -1.0530, -0.3435, -0.7083],
                       [ 0.2801,  0.2255, -1.6126,  1.3294]],
              
                      [[-0.9505, -0.5114, -0.0539, -0.3478],
                       [-0.7235, -1.0530, -0.3435, -0.7083],
                       [ 0.2801,  0.2255, -1.6126,  1.3294]],
              
                      [[-0.9505, -0.5114, -0.0539, -0.3478],
                       [-0.7235, -1.0530, -0.3435, -0.7083],
                       [ 0.2801,  0.2255, -1.6126,  1.3294]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.9505, -0.5114, -0.0539, -0.3478],
                      [-0.7235, -1.0530, -0.3435, -0.7083],
                      [ 0.2801,  0.2255, -1.6126,  1.3294],
                      [-0.9505, -0.5114, -0.0539, -0.3478],
                      [-0.7235, -1.0530, -0.3435, -0.7083],
                      [ 0.2801,  0.2255, -1.6126,  1.3294],
                      [-0.9505, -0.5114, -0.0539, -0.3478],
                      [-0.7235, -1.0530, -0.3435, -0.7083],
                      [ 0.2801,  0.2255, -1.6126,  1.3294],
                      [-0.9505, -0.5114, -0.0539, -0.3478],
                      [-0.7235, -1.0530, -0.3435, -0.7083],
                      [ 0.2801,  0.2255, -1.6126,  1.3294],
                      [-0.9505, -0.5114, -0.0539, -0.3478],
                      [-0.7235, -1.0530, -0.3435, -0.7083],
                      [ 0.2801,  0.2255, -1.6126,  1.3294],
                      [-0.9505, -0.5114, -0.0539, -0.3478],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

333 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
